In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
input_adata_file = "../Datasets/tpm_unstranded.h5ad"
sample_sheet = pd.read_csv("Data/labels.tsv", sep='\t')
clinical = pd.read_csv("Data/clinical.tsv", sep='\t', low_memory=False)

In [3]:
adata = sc.read_h5ad(input_adata_file)
print(adata)

AnnData object with n_obs × n_vars = 2008 × 60660
    obs: 'sample_type'
    var: 'gene_name', 'gene_type'


/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
# Output file path to store subset adata
prefix = input_adata_file.split(".h5ad")[0]
output_file = prefix + "_clinical_features_added.h5ad"

In [5]:
df = pd.DataFrame(
    data=adata.X,
    index=adata.obs_names,         # sample IDs as row index
    columns=adata.var_names        # gene_ids as column names
)

In [6]:
df = df.T

In [7]:
df

,00000c20-b875-4821-8d4b-c50d7a7f9879,000b6b94-572d-4d06-a8f4-2e43829f83d4,0052ae83-7ae5-470a-a125-5cd94a9fa9e9,00810078-a4fc-4e1d-bbdc-e5e0d3080025,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00ad0370-5339-4136-800c-56d4250af0ab,00bc0fe4-481a-4893-b57e-d5113a6bcce2,0163d0f3-46a7-46d2-8ea3-4bf917db2abd,01703923-fdd7-477b-822a-e1eabb0f2404,...,fe1c2811-57fa-4cd8-b5ea-305bb45aabb3,fe20855f-d3ce-4e02-b2dd-3df15eab44bf,fe22447b-79f6-4a9c-9ad8-04fa22b07bd3,fe79c09e-f0ac-4166-b789-33fa1d2fb9d0,fe9b5010-29bb-4e34-97b5-6ce5b512034d,fec22982-c32a-44bd-a1bb-0a377ed028e7,fefb2a51-9eff-464a-bb88-99b886e31698,ff184d74-848e-4a36-b52a-f1304f0996bc,ffc619ed-9dc5-4364-9384-46c9a7fee273,ffec28dd-cc44-4057-bcc3-ccc0cd2331c2
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.15,49.9587,27.3874,94.7370,22.8215,22.1771,49.9587,27.3874,54.7515,13.6987,12.0211,...,33.4408,25.4086,60.7756,77.9371,71.2798,22.2000,9.9647,25.8883,13.9513,47.3889
ENSG00000000005.6,0.0581,0.0482,0.0000,0.0000,0.0000,0.0581,0.0482,0.0000,0.2801,0.0832,...,0.2853,0.0467,0.6407,0.0000,0.1742,0.4031,0.2507,2.5320,0.8981,0.0000
ENSG00000000419.13,23.5219,113.4229,236.5221,64.8944,11.9062,23.5219,113.4229,52.4916,46.5384,87.3003,...,163.6979,62.8583,95.1990,89.5629,41.1897,110.4794,48.0235,89.7125,61.1011,110.4907
ENSG00000000457.14,7.0283,12.6289,4.1846,6.6049,20.8787,7.0283,12.6289,2.9306,9.0620,11.9545,...,13.4889,6.5035,51.5478,6.6064,10.2741,11.7020,10.6522,18.9755,13.4719,8.6045
ENSG00000000460.17,6.2785,12.1395,7.6534,7.2963,4.8143,6.2785,12.1395,5.9356,2.9088,11.3553,...,7.4761,2.6110,41.8473,9.7875,10.3665,12.9434,5.8462,11.2682,4.0671,5.6382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0.0000,0.0000,0.0000,0.0329,0.0000,0.0000,0.0000,0.0000,0.0148,0.0308,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
ENSG00000288670.1,8.8764,10.3222,3.5208,9.4390,23.8585,8.8764,10.3222,6.3307,6.3413,19.9814,...,16.2530,7.7485,41.6076,5.9724,5.6449,6.5441,6.8592,14.3687,9.7689,10.1980
ENSG00000288671.1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [8]:
clinical_columns_to_keep = [
    'case_submitter_id',
    'gender',
    'age_at_index',
    'race',
    'initial_disease_status',
    # 'vital_status',
    'ajcc_pathologic_stage',
    'classification_of_tumor',
    'primary_diagnosis',
    #'tissue_or_organ_of_origin',
    #'ttherapeutic_agents',
    #'treatment_type',
    # 'ajcc_pathologic_n',
    # 'ajcc_pathologic_t',
    # 'ajcc_pathologic_m'
]

In [9]:
columns_to_group = [
    'gender',
    'age_at_index',
    'race',
    'initial_disease_status',
    'ajcc_pathologic_stage',
    'classification_of_tumor',
    'primary_diagnosis',
]

In [10]:
# 1. Build sample-to-case map
sample_to_case = sample_sheet[['File ID', 'Case ID']].dropna()

In [11]:
len(sample_to_case)

2006

In [12]:
sample_to_case = sample_to_case[sample_to_case['File ID'].isin(df.columns)]

In [13]:
len(sample_to_case)

2006

In [14]:
clinical_unique = clinical.drop_duplicates(subset='case_submitter_id')

In [15]:
len(clinical)

5061

In [16]:
len(clinical_unique)

1541

In [17]:
clinical_subset = clinical_unique[clinical_columns_to_keep]

In [18]:
sample_clinical = pd.merge(sample_to_case, clinical_subset,
                           left_on='Case ID',
                           right_on='case_submitter_id',
                           how='left')

In [19]:
sample_clinical.index = sample_clinical['File ID']
sample_clinical = sample_clinical.drop('File ID', axis =1)

In [20]:
# reorder to match OG order
sample_clinical = sample_clinical.loc[df.columns.intersection(sample_clinical.index)]

In [21]:
# Modify normal samples to have NA label for these
clinical_disease_columns = [
    'initial_disease_status',
    'ajcc_pathologic_stage',
    'classification_of_tumor',
    'primary_diagnosis',
]

In [22]:
definitely_cancer_labels = [
    "Primary Tumor",
    "Primary Tumor, Primary Tumor",
    "Primary Tumor, Primary Tumor, Primary Tumor",
    "Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor, Primary Tumor",
    "Metastatic",
    "Recurrent Tumor",
    "Next Generation Cancer Model",
    "Expanded Next Generation Cancer Model"
]

In [23]:
normal_samples = sample_sheet[~sample_sheet['Sample Type'].isin(definitely_cancer_labels)]['File ID']

In [24]:
for col in clinical_disease_columns:
    sample_clinical.loc[normal_samples, col] = "NA/Normal Sample"

In [25]:
adata

AnnData object with n_obs × n_vars = 2008 × 60660
    obs: 'sample_type'
    var: 'gene_name', 'gene_type'

In [26]:
adata.X = df.T

In [27]:
for col in columns_to_group:
   adata.obs[col] = sample_clinical[col].copy()

In [28]:
adata.obs

,sample_type,gender,age_at_index,race,initial_disease_status,ajcc_pathologic_stage,classification_of_tumor,primary_diagnosis
00000c20-b875-4821-8d4b-c50d7a7f9879,Primary Tumor,male,59,white,'--,'--,Prior primary,"Neoplasm, NOS"
000b6b94-572d-4d06-a8f4-2e43829f83d4,Primary Tumor,male,53,white,'--,Stage IIA,primary,"Squamous cell carcinoma, NOS"
0052ae83-7ae5-470a-a125-5cd94a9fa9e9,Primary Tumor,male,61,white,'--,Stage IIIA,primary,"Adenocarcinoma, NOS"
00810078-a4fc-4e1d-bbdc-e5e0d3080025,Primary Tumor,male,57,white,'--,Stage IB,primary,"Squamous cell carcinoma, NOS"
00ad0370-5339-4136-800c-56d4250af0ab,Primary Tumor,male,'--,black or african american,'--,'--,Prior primary,Not Reported
...,...,...,...,...,...,...,...,...
fec22982-c32a-44bd-a1bb-0a377ed028e7,"Primary Tumor, Primary Tumor",NaN,NaN,NaN,NaN,NaN,NaN,NaN
fefb2a51-9eff-464a-bb88-99b886e31698,"Primary Tumor, Primary Tumor",NaN,NaN,NaN,NaN,NaN,NaN,NaN
ff184d74-848e-4a36-b52a-f1304f0996bc,Primary Tumor,male,'--,asian,'--,Stage IIIA,'--,"Adenocarcinoma, NOS"
ffc619ed-9dc5-4364-9384-46c9a7fee273,Solid Tissue Normal,male,'--,asian,NA/Normal Sample,NA/Normal Sample,NA/Normal Sample,NA/Normal Sample


In [29]:
# Save it to an output file
adata.write(output_file)
adata

AnnData object with n_obs × n_vars = 2008 × 60660
    obs: 'sample_type', 'gender', 'age_at_index', 'race', 'initial_disease_status', 'ajcc_pathologic_stage', 'classification_of_tumor', 'primary_diagnosis'
    var: 'gene_name', 'gene_type'